### Notebook for initializing polymer systems using a DPD potential

In [5]:
import matplotlib
import numpy as np  
import gsd, gsd.hoomd 
import hoomd 
import time
import matplotlib_inline
import matplotlib.pyplot as plt
import freud
%matplotlib inline
matplotlib.style.use("ggplot")
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")

## Generate CG trajectory with random walk + DPD workflow

In [43]:
def initialize_snapshot_rand_walk(num_pol, num_mon, density=0.85, bond_length=1.0, buffer=0.1):
    '''
    Create a HOOMD snapshot of a cubic box with the number density given by input parameters.
    '''    
    N = num_pol * num_mon
    L = np.cbrt(N / density)  # Calculate box size based on density
    positions = np.zeros((N, 3))
    for i in range(num_pol):
        start = i * num_mon
        positions[start] = np.random.uniform(low=(-((L/2)-0.5)),high=((L/2)-0.5),size=3)
        for j in range(num_mon - 1):
            delta = np.random.uniform(low=(-bond_length/2),high=(bond_length/2),size=3)
            delta /= np.linalg.norm(delta)*bond_length
            positions[start+j+1] = positions[start+j] + delta
    positions = pbc(positions,[L,L,L])
    bonds = []
    for i in range(num_pol):
        start = i * num_mon
        for j in range(num_mon - 1):
            bonds.append([start + j, start + j + 1])
    bonds = np.array(bonds)
    frame = gsd.hoomd.Frame()
    frame.particles.types = ['A']
    frame.particles.N = N
    frame.particles.position = positions
    frame.bonds.N = len(bonds)
    frame.bonds.group = bonds
    frame.bonds.types = ['b']
    frame.configuration.box = [L, L, L, 0, 0, 0]
    return frame

def pbc(d,box):
    for i in range(3):
        a = d[:,i]
        pos_max = np.max(a)
        pos_min = np.min(a)
        while pos_max > box[i]/2 or pos_min < -box[i]/2:
            a[a < -box[i]/2] += box[i]
            a[a >  box[i]/2] -= box[i]
            pos_max = np.max(a)
            pos_min = np.min(a)
    return d

def check_bond_length_equilibration(snap,num_mon,num_pol,max_bond_length=1.1,min_bond_length=0.9): #todo
    frame_ds = []
    for j in range(num_pol):
        idx = j*num_mon
        d1 = snap.particles.position[idx:idx+num_mon-1] - snap.particles.position[idx+1:idx+num_mon]
        bond_l = np.linalg.norm(pbc(d1,snap.configuration.box),axis=1)
        frame_ds.append(bond_l)
    max_frame_bond_l = np.max(np.array(frame_ds))
    min_frame_bond_l = np.min(np.array(frame_ds))
    print("max: ",max_frame_bond_l," min: ",min_frame_bond_l)
    if max_frame_bond_l <= max_bond_length and min_frame_bond_l >= min_bond_length:
        print("Bond length equillibrated.")
        return True
    if max_frame_bond_l > max_bond_length or min_frame_bond_l < min_bond_length:
        return False

def check_inter_particle_distance(snap,minimum_distance=0.95):
    '''
    Check particle separations.
    
    '''
    positions = snap.particles.position
    box = snap.configuration.box
    aq = freud.locality.AABBQuery(box,positions)
    aq_query = aq.query(
        query_points=positions,
        query_args=dict(r_min=0.0, r_max=minimum_distance, exclude_ii=True),
    )
    nlist = aq_query.toNeighborList()
    if len(nlist)==0:
        print("Inter-particle separation reached.")
        return True
    else:
        return False
        
def run_one(A=1000,gamma=1000,k=1000,num_pol=100,num_mon=10,kT=1.0,r_cut = 1.15,bond_l=1.0,dt=0.001,density=0.01,particle_spacing = 1.1
):
    print(num_pol*num_mon)
    print(f"\nRunning with A={A}, gamma={gamma}, k={k}, "
          f"num_pol={num_pol}, num_mon={num_mon}")
    start_time = time.perf_counter()
    frame = initialize_snapshot_rand_walk(num_pol, num_mon, density=density)
    build_stop = time.perf_counter()
    print("Total build time: ", build_stop-start_time)
    harmonic = hoomd.md.bond.Harmonic()
    harmonic.params["b"] = dict(r0=bond_l, k=k)
    integrator = hoomd.md.Integrator(dt=dt)
    integrator.forces.append(harmonic)
    simulation = hoomd.Simulation(device=hoomd.device.auto_select(), seed=np.random.randint(65535))# TODO seed
    simulation.operations.integrator = integrator 
    simulation.create_state_from_snapshot(frame)
    const_vol = hoomd.md.methods.ConstantVolume(filter=hoomd.filter.All())
    integrator.methods.append(const_vol)
    nlist = hoomd.md.nlist.Cell(buffer=0.4)
    simulation.operations.nlist = nlist
    DPD = hoomd.md.pair.DPD(nlist, default_r_cut=r_cut, kT=kT)
    DPD.params[('A', 'A')] = dict(A=A, gamma=gamma)
    integrator.forces.append(DPD)

    gsd_out = hoomd.write.GSD(
        trigger=hoomd.trigger.Periodic(100),
        mode='wb',
        dynamic=['property', 'momentum'],
        filename='bead-spring-dpd-single-chain.gsd')

    simulation.operations.writers.append(gsd_out)
    
    simulation.run(0)
    simulation.run(100)
    snap=simulation.state.get_snapshot()
    N = num_pol*num_mon
    time_factor = N/90000

    while not check_bond_length_equilibration(snap,num_mon, num_pol,max_bond_length=particle_spacing): #TODO - work with snapshot instead of gsd?
        check_time = time.perf_counter()
        if (check_time-start_time) > 120:
            return num_pol*num_mon, 0
        simulation.run(1000)
        snap=simulation.state.get_snapshot()

    while not check_inter_particle_distance(snap,minimum_distance=0.97):
        check_time = time.perf_counter()
        if (check_time-start_time) > 120:
            return num_pol*num_mon, 0
        simulation.run(1000)
        snap=simulation.state.get_snapshot()
        
    end_time = time.perf_counter()
    gsd_out.flush()
    return num_pol*num_mon, end_time - start_time
    
N,s = run_one(A=5000,gamma=800,k=25000,num_pol=1,num_mon=5) #A=1000 seems fast, cranking up gamma to 800 speeds up, cranking up k more does too,
#no difference between k=10k and 20k
#OK, with these parameters, we're finishing in under 100 steps
print(f"Finished in time = {s:.2f}s")

5

Running with A=5000, gamma=800, k=25000, num_pol=1, num_mon=5
Total build time:  0.0005508750036824495
max:  1.0131161408423388  min:  1.0019358222476706
Bond length equillibrated.
Inter-particle separation reached.
Finished in time = 0.18s


## Isolate last frame from relaxed trajectory

In [44]:
traj = gsd.hoomd.open('bead-spring-dpd-single-chain.gsd','r')
last_frame = traj[-1]
with gsd.hoomd.open('last-dpd-frame.gsd','w') as new_traj:
    new_traj.append(last_frame)

## export last frame to grits fine-graining notebook

## import fine-grained structure

In [ ]:
'''
	<NonbondedForce coulomb14scale="0.5" lj14scale="0.5">
		<Atom type="hs" charge="0.155" sigma="0.0" epsilon="0.0"/>
		<Atom type="ca" charge="-0.155" sigma="0.355" epsilon="0.29288"/>
		<Atom type="s" charge="-0.335" sigma="0.36" epsilon="1.48532"/>
		<Atom type="ha" charge="0.155" sigma="0.242" epsilon="0.12552"/>
		<Atom type="sh" charge="-0.335" sigma="0.36" epsilon="1.7782"/>
	</NonbondedForce>
'''

In [21]:
from flowermd.base import Pack,Lattice, Simulation,BaseHOOMDForcefield,Polymer, forcefield
from flowermd.base.system import System
from flowermd.library import forcefields

In [64]:
class DPD_FF(BaseHOOMDForcefield):
    def __init__(
        self,
        epsilon,
        A,
        gamma,
        kT,
        r_cut
    ):
        self.epsilon = epsilon
        self.gamma = gamma
        self.A = A
        self.kT = kT
        self.r_cut = r_cut
        hoomd_forces = self._create_forcefield()
        super(DPD_FF, self).__init__(hoomd_forces)

    def _create_forcefield(self):
        forces = []
        # Bonds
        bond = hoomd.md.bond.Harmonic()
        bond.params["C-H"] = dict(k="307105.6", r0="1.108")
        bond.params["C-S"] = dict(k="209200.0", r0="1.176") #might need atom type mapping
        bond.params["H-S"] = dict(k="229283.2", r0="1.1336")
        #bond.params["C-S"] = dict(k="209200.0", r0="0.176")
        bond.params["C-C"] = dict(k="392459.2", r0="1.14")
        bond.params["H-C"] = dict(k="307105.6", r0="1.108")
        #bond.params["SH-CA"] = dict(k="209200.0", r0="0.174")
        forces.append(bond)
        # Angles
        '''
        angle = hoomd.md.angle.Harmonic()
        angle.params["C-S-C"] = dict(k="627.6", t0="1.805")
        angle.params["C-S-H"] = dict(k="418.4", t0="1.67551608191")
        angle.params["C-C-S"] = dict(k="711.28", t0="2.08392312688")
        angle.params["C-C-C"] = dict(k="527.184", t0="2.09439510239")
        angle.params["C-C-S"] = dict(k="585.76", t0="2.09439510239")
        angle.params["S-C-C"] = dict(k="585.76", t0="2.09439510239")
        angle.params["C-C-H"] = dict(k="292.88", t0="2.09439510239")
        angle.params["S-C-C"] = dict(k="711.28", t0="2.08392312688")
        forces.append(angle)
        '''
        # DPD Pairs
        nlist = hoomd.md.nlist.Cell(buffer=0.40, exclusions=["body"])
        dpd = hoomd.md.pair.DPD(nlist=nlist,kT=self.kT,default_r_cut=self.r_cut)
        for pair in [
            ("H","H"),
            ("H","C"),
            ("H","S"),
            ("C","C"),
            ("S","H"),
            ("C","H"),
            ("C","S"),
            ("S","S")
        ]:
            dpd.params[pair] = dict(A=self.A,gamma=self.gamma)
            dpd.params[pair].r_cut = self.r_cut
        forces.append(dpd)
        return forces

In [65]:
ff = DPD_FF(epsilon=1.0,A=5000,gamma=800,kT=1.0,r_cut=1.15)
sim = Simulation(
    initial_state='pps-from-dpd-single-chain.gsd',
    forcefield=ff.hoomd_forces,
    dt=0.001,
    gsd_write_freq=int(100),
    gsd_file_name='trajectory.gsd',
    log_write_freq=int(100),
    log_file_name='log.txt')

sim.save_restart_gsd()
sim.run_NVT(n_steps=1e4, kT=1.0, tau_kt=10*sim.dt)
sim.flush_writers()

Initializing simulation state from a GSD file.
Step 100 of 10000; TPS: 611.66; ETA: 0.3 minutes
Step 200 of 10000; TPS: 1218.29; ETA: 0.1 minutes
Step 300 of 10000; TPS: 1822.19; ETA: 0.1 minutes
Step 400 of 10000; TPS: 2422.95; ETA: 0.1 minutes
Step 500 of 10000; TPS: 3020.35; ETA: 0.1 minutes
Step 600 of 10000; TPS: 3599.22; ETA: 0.0 minutes
Step 700 of 10000; TPS: 4187.5; ETA: 0.0 minutes
Step 800 of 10000; TPS: 4773.1; ETA: 0.0 minutes
Step 900 of 10000; TPS: 5355.07; ETA: 0.0 minutes
Step 1000 of 10000; TPS: 5933.98; ETA: 0.0 minutes
Step 1100 of 10000; TPS: 6509.76; ETA: 0.0 minutes
Step 1200 of 10000; TPS: 7073.68; ETA: 0.0 minutes
Step 1300 of 10000; TPS: 7638.57; ETA: 0.0 minutes
Step 1400 of 10000; TPS: 8203.59; ETA: 0.0 minutes
Step 1500 of 10000; TPS: 8759.79; ETA: 0.0 minutes
Step 1600 of 10000; TPS: 9310.72; ETA: 0.0 minutes
Step 1700 of 10000; TPS: 9859.24; ETA: 0.0 minutes
Step 1800 of 10000; TPS: 10408.23; ETA: 0.0 minutes
Step 1900 of 10000; TPS: 10954.23; ETA: 0.0 mi